In [1]:
# pip install ipython torch python-dotenv opencv-python matplotlib roboflow ultralytics mss numpy pyautogui math

from IPython import display
import torch
from dotenv import load_dotenv
import shutil
import cv2
import os
import roboflow
import ultralytics
from ultralytics import YOLO
import mss
import numpy as np
import time
import pyautogui
from datetime import datetime
from math import sqrt

In [2]:
display.clear_output()
ultralytics.checks()

Ultralytics 8.3.133  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
Setup complete  (12 CPUs, 15.9 GB RAM, 246.2/931.5 GB disk)


In [3]:
# Em apple sillicon, verificar se o MPS está disponível
print(torch.backends.mps.is_available())

False


In [4]:
# Carregar variáveis do ficheiro .env
load_dotenv(override=True)

# Obter variáveis
api_key = os.getenv("ROBOFLOW_API_KEY")
workspace = os.getenv("ROBOFLOW_WORKSPACE")
project_name = os.getenv("ROBOFLOW_PROJECT")
version = int(os.getenv("ROBOFLOW_VERSION", "1"))

In [ ]:
# o treino do modelo esta no google colab (https://colab.research.google.com/drive/1Lr8e85GB5lLbuTxvJtXceK6WwTwW15lt?usp=sharing)

###### MODELO RELACIONADO COM OS OBJETIVOS DO NIVEL ATUAL

In [ ]:
# o treino do modelo esta no google colab (https://colab.research.google.com/drive/1Lr8e85GB5lLbuTxvJtXceK6WwTwW15lt?usp=sharing)

# 🎮Testar no jogo, o modelo

###### CONFIGURAÇÕES GERAIS

In [7]:
model = YOLO("runs/detect/train2/weights/best.pt") # Carrega o modelo YOLO treinado
modelGoal = YOLO("runs/detect/train/weights/best.pt") # Carrega o modelo YOLO treinado do objetivo
screen_width, screen_height = pyautogui.size() # Tamanho total do ecrã (para eventualmente converter coordenadas para cliques)

# Região do tabuleiro (ajustável via variáveis de ambiente)
BOARD_REGION = {
    "left": int(os.getenv("BOARD_REGION_LEFT", "0")),     # posição X do canto superior esquerdo
    "top": int(os.getenv("BOARD_REGION_TOP", "0")),      # posição Y do canto superior esquerdo
    "width": int(os.getenv("BOARD_REGION_WIDTH", "0")),    # largura da região
    "height": int(os.getenv("BOARD_REGION_HEIGHT", "0"))    # altura da região
}

# Número de linhas e colunas no tabuleiro (ex: 7x7)
ROWS = int(os.getenv("ROWS_GRID", "8"))
COLS = int(os.getenv("COLS_GRID", "8"))

# Região do objetivo (ajustável via variáveis de ambiente)
GOAL_REGION = {
    "left": int(os.getenv("GOAL_REGION_LEFT", "0")),     # posição X do canto superior esquerdo
    "top": int(os.getenv("GOAL_REGION_TOP", "0")),      # posição Y do canto superior esquerdo
    "width": int(os.getenv("GOAL_REGION_WIDTH", "0")),    # largura da região
    "height": int(os.getenv("GOAL_REGION_HEIGHT", "0"))    # altura da região
}

# Garante que as pastas de output existem
os.makedirs("captured_images", exist_ok=True)
os.makedirs("captured_goals", exist_ok=True)
os.makedirs("detections", exist_ok=True)

###### FUNÇÃO DE CAPTURA DE IMAGEM

In [8]:
# função que captura só a região do tabuleiro e devolve a imagem
def capture_board_region():
    # Captura a imagem da região definida como o tabuleiro
    with mss.mss() as sct:
        screenshot = sct.grab(BOARD_REGION)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_height, img_width, _ = img.shape
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

        # Guardar imagem opcionalmente
        img_path = os.path.join('captured_images', f"capture_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height # width, height

###### FUNCAO DE CAPTURA DA REGIAO DOS OBJETIVOS

In [9]:
# função que captura só a região dos objetivos e devolve a imagem
def capture_goal_region():
    # Captura a imagem da região definida como os objetivos
    with mss.mss() as sct:
        screenshot = sct.grab(GOAL_REGION)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_height, img_width, _ = img.shape
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

        # Guardar imagem opcionalmente
        img_path = os.path.join('captured_goals', f"goal_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height # width, height

###### IDENTIFICAR PECAS ADJACENTES E POSSIVEIS COMBINACOES

In [10]:
def sugerir_melhor_jogada(grid):
    global ultima_jogada_objetivo
    ROWS, COLS = len(grid), len(grid[0])
    melhores_trocas = []
    max_combo = 0
    troca_com_color_bomb = None

    os.makedirs("debug_objetivos_crop", exist_ok=True)
    imgGoal, timestampGoal, img_widthGoal, img_heightGoal = capture_goal_region()

    # Detetar números (modelGoal)
    resultsGoal = modelGoal.predict(source=imgGoal, conf=0.1)
    caixas_numeros = resultsGoal[0].boxes.xyxy.cpu().numpy()
    classes_numeros = resultsGoal[0].boxes.cls.cpu().numpy()
    nomes_numeros = modelGoal.names

    # Detetar ícones (model)
    resultsIcones = model.predict(source=imgGoal, conf=0.1)
    caixas_icones = resultsIcones[0].boxes.xyxy.cpu().numpy()
    classes_icones = resultsIcones[0].boxes.cls.cpu().numpy()
    nomes_icones = model.names

    for j, classe_ico in enumerate(classes_icones):
        nome = nomes_icones[int(classe_ico)]
        print(f"[DEBUG] Ícone detetado: {nome}")

    objetivos_detectados = []
    print(f"[DEBUG] Objetivos detetados: {objetivos_detectados}")

    # Preencher a lista com os dígitos detetados pelo YOLO
    digitos = []
    for i, (caixa_num, classe_num) in enumerate(zip(caixas_numeros, classes_numeros)):
        x1n, y1n, x2n, y2n = caixa_num.astype(int)
        centro_x = (x1n + x2n) // 2
        centro_y = (y1n + y2n) // 2
        digito = nomes_numeros[int(classe_num)]
        digitos.append({"x": centro_x, "y": centro_y, "valor": digito, "caixa": (x1n, y1n, x2n, y2n)})

    def distancia(p1, p2):
        return sqrt((p1["x"] - p2["x"])**2 + (p1["y"] - p2["y"])**2)

    def agrupar_digitos_em_numeros(digitos, max_dx=35, max_dy=12):
        grupos = []
        usados = set()
        for i, d1 in enumerate(digitos):
            if i in usados:
                continue
            grupo = [d1]
            usados.add(i)
            for j, d2 in enumerate(digitos):
                if j in usados:
                    continue
                mesma_linha = abs(d1["y"] - d2["y"]) <= max_dy
                distancia_x = abs(d1["x"] - d2["x"])
                if mesma_linha and distancia_x <= max_dx:
                    grupo.append(d2)
                    usados.add(j)
            grupo_ordenado = sorted(grupo, key=lambda d: d["x"])
            grupos.append(grupo_ordenado)
        return sorted(grupos, key=lambda g: sum(d["y"] for d in g) / len(g))

    grupos_digitos = agrupar_digitos_em_numeros(digitos)

    usados = set()
    for j, (caixa_ico, classe_ico) in enumerate(zip(caixas_icones, classes_icones)):
        x1i, y1i, x2i, y2i = caixa_ico.astype(int)
        centro_ico_x = (x1i + x2i) // 2
        centro_ico_y = (y1i + y2i) // 2
        nome_icone = nomes_icones[int(classe_ico)]
        grupo_candidato = None
        menor_distancia_y = float("inf")
        for i, grupo in enumerate(grupos_digitos):
            if i in usados:
                continue
            centro_grupo_x = sum(d["x"] for d in grupo) // len(grupo)
            centro_grupo_y = sum(d["y"] for d in grupo) // len(grupo)
            alinhado = abs(centro_grupo_x - centro_ico_x) <= 30
            abaixo = centro_grupo_y > centro_ico_y
            distancia_y = centro_grupo_y - centro_ico_y
            if alinhado and abaixo and distancia_y < menor_distancia_y:
                grupo_candidato = grupo
                menor_distancia_y = distancia_y
                idx_candidato = i
        if grupo_candidato:
            # Número válido → adiciona como objetivo
            usados.add(idx_candidato)
            numero = ''.join(d["valor"] for d in grupo_candidato)
            x = [d["x"] for d in grupo_candidato]
            y = [d["y"] for d in grupo_candidato]
            print(f"[DEBUG] Grupo de dígitos: {numero} -> x: {x}, y: {y}")
            print(f"[DEBUG] Detetado objetivo: {nome_icone} → {numero}")

            # Guardar imagem de debug com ícone e número
            x1_crop = min(x1i, grupo_candidato[0]["caixa"][0]) - 10
            y1_crop = min(y1i, grupo_candidato[0]["caixa"][1]) - 10
            x2_crop = max(x2i, grupo_candidato[-1]["caixa"][2]) + 10
            y2_crop = max(y2i, grupo_candidato[-1]["caixa"][3]) + 10

            x1_crop = max(0, x1_crop)
            y1_crop = max(0, y1_crop)
            x2_crop = min(img_widthGoal, x2_crop)
            y2_crop = min(img_heightGoal, y2_crop)

            crop_img = imgGoal[y1_crop:y2_crop, x1_crop:x2_crop].copy()
            cv2.imwrite(f"debug_objetivos_crop/{nome_icone}_{numero}_{x1i}_{y1i}.jpg", crop_img)

            objetivos_detectados.append({
                "cor": nome_icone,
                "quantidade": numero,
                "caixa": grupo_candidato[0]["caixa"]
            })

            if numero.lower() == "check":
                print(f"[✅ Ignorado: objetivo '{nome_icone}' já concluído (quantidade = check)]")
                continue

        else:
            # ⚠️ O ícone está visível mas sem número → provavelmente com o check (✓) → IGNORAR
            print(f"[✅ Ignorado: objetivo '{nome_icone}' já concluído (sem número visível)]")
            continue



    # Obter lista de cores com números detetados
    cores_com_numeros = [
        obj["cor"]
        for obj in objetivos_detectados
        if obj["quantidade"].isdigit() and int(obj["quantidade"]) > 0
    ]

    objetivos_filtrados = []
    for obj in objetivos_detectados:
        if obj["quantidade"].isdigit() and int(obj["quantidade"]) > 0:
            objetivos_filtrados.append(obj)
        elif obj["quantidade"].isdigit() is False:
            if obj["cor"] in cores_com_numeros:
                print(f"[DEBUG] Quantidade ilegível para {obj['cor']} — ignorando porque outros têm número")
            else:
                print(f"[DEBUG] Quantidade ilegível para {obj['cor']} — mantendo ativo por ser o único")
                objetivos_filtrados.append(obj)

    objetivos_detectados = objetivos_filtrados

    print("[DEBUG] Objetivos ativos após validação:", [f"{obj['cor']} → {obj['quantidade']}" for obj in objetivos_detectados])


    def posicoes_sequencia(g, r, c, cor):
        posicoes = [(r, c)]
        i = c - 1
        while i >= 0 and g[r][i] == cor:
            posicoes.append((r, i))
            i -= 1
        i = c + 1
        while i < COLS and g[r][i] == cor:
            posicoes.append((r, i))
            i += 1
        if len(posicoes) >= 3:
            return posicoes

        posicoes = [(r, c)]
        i = r - 1
        while i >= 0 and g[i][c] == cor:
            posicoes.append((i, c))
            i -= 1
        i = r + 1
        while i < ROWS and g[i][c] == cor:
            posicoes.append((i, c))
            i += 1
        if len(posicoes) >= 3:
            return posicoes
        return []

    def seq_inclui_qualquer_doce_objetivo(posicoes, objetivos):
        for (rr, cc) in posicoes:
            if any(grid[rr][cc] == obj["cor"] for obj in objetivos):
                return True
        return False

    jogadas_objetivos = []
    todas_jogadas = []

    for r in range(ROWS):
        for c in range(COLS):
            if grid[r][c] is None:
                continue
            for dr, dc in [(0, 1), (1, 0)]:
                nr, nc = r + dr, c + dc
                if nr >= ROWS or nc >= COLS or grid[nr][nc] is None:
                    continue

                if grid[r][c] == "color-bomb" or grid[nr][nc] == "color-bomb":
                    troca_com_color_bomb = ((r, c), (nr, nc))

                cor1, cor2 = grid[r][c], grid[nr][nc]
                grid[r][c], grid[nr][nc] = grid[nr][nc], grid[r][c]

                posicoes_r_c = posicoes_sequencia(grid, r, c, cor2) or []
                posicoes_nr_nc = posicoes_sequencia(grid, nr, nc, cor1) or []

                score = max(len(posicoes_r_c), len(posicoes_nr_nc))

                if score >= 3:
                    if score > max_combo:
                        max_combo = score
                        melhores_trocas = [((r, c), (nr, nc))]
                    elif score == max_combo:
                        melhores_trocas.append(((r, c), (nr, nc)))

                    doces_na_seq = [grid[rr][cc] for (rr, cc) in posicoes_r_c + posicoes_nr_nc]
                    objetivos_na_seq = [obj for obj in objetivos_detectados if obj["cor"] in doces_na_seq and obj["quantidade"].isdigit()]
                    if objetivos_na_seq:
                        objetivos_validos = [obj for obj in objetivos_na_seq if obj["quantidade"].isdigit()]
                        if objetivos_validos:
                            prioridade = max(int(obj["quantidade"]) for obj in objetivos_validos)
                            jogadas_objetivos.append(((r, c), (nr, nc), prioridade, score))
                        else:
                            todas_jogadas.append(((r, c), (nr, nc), 0, score))


                grid[r][c], grid[nr][nc] = grid[nr][nc], grid[r][c]

    if troca_com_color_bomb:
        print(f"\n💡 Jogada com 'color-bomb': {troca_com_color_bomb} (prioritária)")
        return troca_com_color_bomb, 99

    if jogadas_objetivos:
        objetivos_ordenados = sorted(set(j[2] for j in jogadas_objetivos), reverse=True)
        jogada_encontrada = None

        for idx, prioridade_atual in enumerate(objetivos_ordenados):
            jogadas_para_prioridade = [j for j in jogadas_objetivos if j[2] == prioridade_atual]
            jogadas_para_prioridade.sort(key=lambda x: x[3], reverse=True)

            print(f"\n🔍 Tentativa com objetivo de prioridade {prioridade_atual}")

            for jogada in jogadas_para_prioridade:
                jogada_coords = jogada[:2]
                if jogada_coords != ultima_jogada_objetivo:
                    ultima_jogada_objetivo = jogada_coords
                    tipo = "principal" if idx == 0 else "secundário"
                    print(f"\n💡 Jogada de objetivo {tipo}: {jogada_coords} com prioridade {prioridade_atual}")
                    return jogada_coords, prioridade_atual

            if jogada_encontrada is None:
                jogada_encontrada = jogadas_para_prioridade[0]

        if jogada_encontrada:
            print(f"\n🔁 Todas jogadas são repetidas. Reutilizar: {jogada_encontrada[:2]} com prioridade {jogada_encontrada[2]}")
            return jogada_encontrada[:2], jogada_encontrada[2]

    if todas_jogadas:
        todas_jogadas.sort(key=lambda x: x[3], reverse=True)
        print(f"\n💡 Jogada comum melhor: {todas_jogadas[0][:2]} com combo {todas_jogadas[0][3]}")
        return todas_jogadas[0][:2], todas_jogadas[0][3]

    if melhores_trocas:
        print(f"\n💡 Melhor jogada: {melhores_trocas[0]} que forma grupo de {max_combo}")
        return melhores_trocas[0], max_combo

    print("\n⚠️ Nenhuma jogada possível encontrada.")
    return None, 0

###### LOOP PRINCIPAL

In [12]:
ultima_jogada_objetivo = None

while True:
    # Captura o tabuleiro
    img, timestamp, img_width, img_height = capture_board_region()

    # Previsão com YOLO (doces)
    results = model.predict(source=img, conf=0.1)
    detections = results[0].boxes.xyxy

    # Define dimensões da grelha
    cell_width = img_width // COLS
    cell_height = img_height // ROWS

    grid = [[None for _ in range(COLS)] for _ in range(ROWS)]

    if len(detections) > 0:
        print(f"\n✅ Detetou {len(detections)} objetos.")

        # Guarda imagem anotada
        annotated_frame = results[0].plot()
        result_path = os.path.join('detections', f"result_{timestamp}.jpg")
        cv2.imwrite(result_path, annotated_frame)

        # Preenche a grelha com base nas deteções
        for i, (x1, y1, x2, y2) in enumerate(detections.tolist()):
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            col = center_x // cell_width
            row = center_y // cell_height

            class_id = int(results[0].boxes.cls[i].item())
            class_name = model.names[class_id]

            if 0 <= row < ROWS and 0 <= col < COLS:
                grid[row][col] = class_name

        # Imprime o mapa atual
        print("📌 Mapa do tabuleiro:")
        for row in grid:
            print(row)

        # Sugere jogada com base no modelo de objetivos
        print("\n🔄 Combinações possíveis:")
        melhor_troca, tamanho_grupo = sugerir_melhor_jogada(grid)

        if melhor_troca:
            (r1, c1), (r2, c2) = melhor_troca

            # ✅ Nova captura antes da jogada
            img_ref, timestamp_ref, _, _ = capture_board_region()
            results_ref = model.predict(source=img_ref, conf=0.1)
            detections_ref = results_ref[0].boxes.xyxy

            grid_ref = [[None for _ in range(COLS)] for _ in range(ROWS)]
            for i, (x1, y1, x2, y2) in enumerate(detections_ref.tolist()):
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                col = center_x // cell_width
                row = center_y // cell_height

                class_id = int(results_ref[0].boxes.cls[i].item())
                class_name = model.names[class_id]

                if 0 <= row < ROWS and 0 <= col < COLS:
                    grid_ref[row][col] = class_name

            # Mostra grelha atualizada antes da jogada
            print("📌 Tabuleiro atualizado antes da jogada:")
            for row in grid_ref:
                print(row)

            # Converte coordenadas para o ecrã
            def cell_to_screen(row, col):
                x = BOARD_REGION["left"] + col * cell_width + cell_width // 2
                y = BOARD_REGION["top"] + row * cell_height + cell_height // 2
                return x, y

            x1, y1 = cell_to_screen(r1, c1)
            x2, y2 = cell_to_screen(r2, c2)

            print(f"🖱️ A mover para ({r1}, {c1}) -> ({r2}, {c2})")
            pyautogui.moveTo(x1, y1, duration=0.3)
            pyautogui.click()
            time.sleep(0.2)
            pyautogui.moveTo(x2, y2, duration=0.3)
            pyautogui.click()

        else:
            print("⚠️ Nenhuma jogada possível para clicar.")

    else:
        print("⚠️ Nenhuma deteção feita no tabuleiro.")

    time.sleep(5)  # tempo ajustável entre jogadas


0: 640x640 13 blues, 9 greens, 13 purples, 14 reds, 7 yellows, 281.4ms
Speed: 2.3ms preprocess, 281.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

✅ Detetou 56 objetos.
📌 Mapa do tabuleiro:
[None, 'purple', 'yellow', 'purple', 'purple', 'green', 'red', None]
['red', 'yellow', 'purple', 'red', 'blue', 'red', 'blue', 'green']
[None, 'blue', 'red', 'green', 'purple', 'yellow', 'blue', None]
['green', 'red', 'green', 'blue', 'red', 'yellow', 'red', 'green']
['green', 'yellow', 'purple', 'red', 'blue', 'green', 'purple', 'green']
[None, 'blue', 'yellow', 'blue', 'red', 'purple', 'purple', None]
['red', 'red', 'purple', 'purple', 'blue', 'yellow', 'blue', 'red']
[None, 'blue', 'purple', 'blue', 'purple', 'red', 'blue', None]

🔄 Combinações possíveis:

0: 640x512 1 1, 1 2, 1 3, 1 6, 224.6ms
Speed: 1.6ms preprocess, 224.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 512)

0: 640x512 1 blocker, 1 blue, 1 red, 2 yellows, 221.9ms
Speed: 1.7ms preprocess, 2

KeyboardInterrupt: 